In [1]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd
from os import listdir
import cv2
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image  as mpimg

In [ ]:
/content/drive/MyDrive/data/data/with_mask

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
print("The number of images with facemask labelled 'yes':",len(os.listdir('/content/drive/MyDrive/data/data/with_mask')))
print("The number of images with facemask labelled 'no':",len(os.listdir('/content/drive/MyDrive/data/data/without_mask')))

The number of images with facemask labelled 'yes': 690
The number of images with facemask labelled 'no': 686


In [14]:
def data_summary(main_path):
    
    yes_path = main_path+'with_mask'
    no_path = main_path+'without_mask'
        
    # number of files (images) that are in the the folder named 'yes' that represent tumorous (positive) examples
    m_pos = len(listdir(yes_path))
    # number of files (images) that are in the the folder named 'no' that represent non-tumorous (negative) examples
    m_neg = len(listdir(no_path))
    # number of all examples
    m = (m_pos+m_neg)
    
    pos_prec = (m_pos* 100.0)/ m
    neg_prec = (m_neg* 100.0)/ m
    
    print(f"Number of examples: {m}")
    print(f"Percentage of positive examples: {pos_prec}%, number of pos examples: {m_pos}") 
    print(f"Percentage of negative examples: {neg_prec}%, number of neg examples: {m_neg}") 
    
augmented_data_path = '/content/drive/MyDrive/data/data/'    
data_summary(augmented_data_path)

Number of examples: 1376
Percentage of positive examples: 50.145348837209305%, number of pos examples: 690
Percentage of negative examples: 49.854651162790695%, number of neg examples: 686


In [19]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    dataset = []
    
    for unitData in os.listdir(SOURCE):
        data = SOURCE + unitData
        if(os.path.getsize(data) > 0):
            dataset.append(unitData)
        else:
            print('Skipped ' + unitData)
            print('Invalid file i.e zero size')
    
    train_set_length = int(len(dataset) * SPLIT_SIZE)
    test_set_length = int(len(dataset) - train_set_length)
    shuffled_set = random.sample(dataset, len(dataset))
    train_set = dataset[0:train_set_length]
    test_set = dataset[-test_set_length:]
       
    for unitData in train_set:
        temp_train_set = SOURCE + unitData
        final_train_set = TRAINING + unitData
        copyfile(temp_train_set, final_train_set)
    
    for unitData in test_set:
        temp_test_set = SOURCE + unitData
        final_test_set = TESTING + unitData
        copyfile(temp_test_set, final_test_set)
        
        
YES_SOURCE_DIR = "/content/drive/MyDrive/data/data/augmented data1/val/with_mask/"
TRAINING_YES_DIR = "/content/drive/MyDrive/data/data/augmented data1/train/with_mask/"
TESTING_YES_DIR = "/content/drive/MyDrive/data/data/augmented data1/test/with_mask/"
NO_SOURCE_DIR = "/content/drive/MyDrive/data/data/augmented data1/val/without_mask/"
TRAINING_NO_DIR = "/content/drive/MyDrive/data/data/augmented data1/train/without_mask/"
TESTING_NO_DIR = "/content/drive/MyDrive/data/data/augmented data1/test/without_mask/"
split_size = .8
split_data(YES_SOURCE_DIR, TRAINING_YES_DIR, TESTING_YES_DIR, split_size)
split_data(NO_SOURCE_DIR, TRAINING_NO_DIR, TESTING_NO_DIR, split_size)

In [21]:
print("The number of images with facemask in the training set labelled 'yes':", len(os.listdir('/content/drive/MyDrive/data/data/augmented data1/train/with_mask/')))
print("The number of images with facemask in the test set labelled 'yes':", len(os.listdir('/content/drive/MyDrive/data/data/augmented data1/test/with_mask/')))
print("The number of images without facemask in the training set labelled 'no':", len(os.listdir('/content/drive/MyDrive/data/data/augmented data1/train/without_mask/')))
print("The number of images without facemask in the test set labelled 'no':", len(os.listdir('/content/drive/MyDrive/data/data/augmented data1/test/without_mask/')))

The number of images with facemask in the training set labelled 'yes': 56
The number of images with facemask in the test set labelled 'yes': 112
The number of images without facemask in the training set labelled 'no': 216
The number of images without facemask in the test set labelled 'no': 112


In [22]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(100, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [25]:
TRAINING_DIR = "/content/drive/MyDrive/data/data/augmented data1/train/"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))
VALIDATION_DIR = "/content/drive/MyDrive/data/data/augmented data1/test/"
validation_datagen = ImageDataGenerator(rescale=1.0/255)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

Found 359 images belonging to 2 classes.
Found 224 images belonging to 2 classes.


In [26]:
history = model.fit_generator(train_generator,
                              epochs=30,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
36/36 [==============================] - 170s 5s/step - loss: 0.5404 - acc: 0.8384 - val_loss: 0.8037 - val_acc: 0.5179
INFO:tensorflow:Assets written to: model-001.model/assets
Epoch 2/30
36/36 [==============================] - 38s 1s/step - loss: 0.3932 - acc: 0.8440 - val_loss: 0.7530 - val_acc: 0.5134
INFO:tensorflow:Assets written to: model-002.model/assets
Epoch 3/30
36/36 [==============================] - 38s 1s/step - loss: 0.2750 - acc: 0.8802 - val_loss: 0.3418 - val_acc: 0.8705
INFO:tensorflow:Assets written to: model-003.model/assets
Epoch 4/30
36/36 [==============================] - 38s 1s/step - loss: 0.2596 - acc: 0.9025 - val_loss: 0.3338 - val_acc: 0.8795
INFO:tensorflow:Assets written to: model-004.model/assets
Epoch 5/30
36/36 [==============================] - 38s 1s/step - loss: 0.3746 - acc: 0.8412 - val_loss: 0.3469 - val_acc: 0.8795
Epoch 6/30
36/36 [==============================] - 38s 1s/step - loss: 0.2569 - acc: 0.8969 - val_loss: 0.2219 - val

In [27]:
face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [29]:
labels_dict={0:'without_mask',1:'with_mask'}
color_dict={0:(0,0,255),1:(0,255,0)}

size = 4
webcam = cv2.VideoCapture(0) #Use camera 0

# We load the xml file
classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while True:
    (rval, im) = webcam.read()
    im=cv2.flip(im,1,1) #Flip to act as a mirror

    # Resize the image to speed up detection
    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))

    # detect MultiScale / faces 
    faces = classifier.detectMultiScale(mini)

    # Draw rectangles around each face
    for f in faces:
        (x, y, w, h) = [v * size for v in f] #Scale the shapesize backup
        #Save just the rectangle faces in SubRecFaces
        face_img = im[y:y+h, x:x+w]
        resized=cv2.resize(face_img,(150,150))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        #print(result)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    # Show the image
    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    # if Esc key is press then break out of the loop 
    if key == 27: #The Esc key
        break
# Stop video
webcam.release()

# Close all started windows
cv2.destroyAllWindows()

AttributeError: ignored